In [2]:
import pandas as pd 
import numpy as np

### Parsing the incredibly poorly written LDNe output

The output is a mixture of different lengths whitespaces and symbols, and so requires some parsing....

In [8]:
phase3ldne_list = pd.read_csv("../data/Phase3.LDNe.list")
collections = phase3ldne_list['pop']
phase3ldne_list.shape

(63, 1)

In [9]:
phase3ldne_list.head()

,pop
0,AG1000G-AO.Luanda.2009.coluzzii
1,AG1000G-BF-A.Bana.2012.coluzzii
2,AG1000G-BF-A.Bana.2012.gambiae
3,AG1000G-BF-A.Pala.2012.coluzzii
4,AG1000G-BF-A.Pala.2012.gambiae


In [10]:
chroms = ['3L']

Ne_Ag = dict()
Ne_chrom = dict()

for name in collections:
    for chrom in chroms:
        df = pd.read_csv(f"LDNe/Ag_LDNe_{name}.{chrom}.out", header=None, sep='\n')
        df = df[0].str.split('\s\=\s', expand=True) #split first column using equal sign
        df.columns = ['one', 'two']
        start = np.where(df.one.str.contains('Harmonic Mean'))[0][0] #find start of results
        end = len(df)-3
        df = df.iloc[start:end]                                           #subset 
        
        #loop through rows and remove multiple whitespaces to one whitespace only, then split to new columns
        res=pd.DataFrame()
        for i in range(5):
            results = ' '.join(df.iloc[i,1].split()).split()
            res = res.append(pd.Series(results),ignore_index=True)
        
        #get columns which have estimates for CIs (poorly parsed)
        df2 = df.iloc[6:9,]
        jack = df.iloc[9,0]
        df2.one = df2.one.str.replace("*", "")
        df2 = df2.one.str.split(expand=True).reset_index(drop=True)
        
        #loop through rows and remove multiple whitespaces to one whitespace only, then split to new columns
        for i in range(len(df2)):
            df2.iloc[i,0] = ' '.join(df2.iloc[i,0].split())
        
        #extract specific estimates,works only because all files are parsed identically, not ideal
        para_1 = df2.loc[0,1:4]
        para_2 = df2.loc[1,0:3]
        jack_1 = df2.loc[2,3:7]
        jack_2 = pd.Series(jack.split())
        cols = df.iloc[:5]['one']
        cols = cols.append(pd.Series(['Parametric CI - lower', 
                              'Parametric CI - upper', 
                              'Jackknife CI - lower', 
                              'Jackknife CI - upper']), ignore_index=True)
        
        #append the  estimates to the results 
        df = res.append(para_1.reset_index(drop=True)).append(para_2.reset_index(drop=True)).append(jack_1.reset_index(drop=True)).append(jack_2, ignore_index=True)
        
        #join results to the parameter column and change column names 
        final_results = pd.concat([cols,df], axis=1)
        final_results.columns = ['Parameter', 
                         'minAF_0.05', 
                         'minAF_0.02', 
                         'minAF_0.01', 
                         'minAF_0+']
        
        #store results in dict and then save within nested dict
        Ne_chrom[chrom] = final_results.iloc[:,:2] ### .iloc[:,:2] for 1 allele frequency only!
        
    Ne_Ag[name] = dict(Ne_chrom)

/home/sanj/apps/anaconda3/envs/mapping/lib/python3.8/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [11]:
Ne = pd.DataFrame()

# transpose, add population and chromosome columns, then re-order columns, and combine all tables for each
# pop into one big one
for name in collections:
    for chrom in chroms:
        
        Ne_Ag[name][chrom] = Ne_Ag[name][chrom].set_index('Parameter').T
        
        Ne_Ag[name][chrom]['name'] = name
        Ne_Ag[name][chrom]['chrom'] = chrom

                # get a list of columns
        cols = list(Ne_Ag[name][chrom])
        # move the column to head of list using index, pop and insert
        cols.insert(0, cols.pop(cols.index('name')))
        Ne_Ag[name][chrom] = Ne_Ag[name][chrom].loc[:, cols]
        cols = list(Ne_Ag[name][chrom])
        cols.insert(0, cols.pop(cols.index('chrom')))
        Ne_Ag[name][chrom] = Ne_Ag[name][chrom].loc[:, cols]
        Ne_Ag[name][chrom]
        
        Ne = pd.concat([Ne, Ne_Ag[name][chrom]])

#change names removing whitespace
Ne.columns = ['chrom', 'pop', 'sample_size', 'independent_comparisons', 
                      'overall_r^2', 'expected_r^2', 'Ne_estimate', 'Parametric_CI_lower',
                     'Parametric_CI_upper', 'Jackknife_CI_lower', 'Jackknife_CI_upper']

In [12]:
Ne

,chrom,pop,sample_size,independent_comparisons,overall_r^2,expected_r^2,Ne_estimate,Parametric_CI_lower,Parametric_CI_upper,Jackknife_CI_lower,Jackknife_CI_upper
minAF_0.05,3L,AG1000G-AO.Luanda.2009.coluzzii,81.0,188382834,0.014563,0.012832,190.4,190.1,190.8,110.4,519.7
minAF_0.05,3L,AG1000G-BF-A.Bana.2012.coluzzii,42.0,224071764,0.025595,0.025618,Infinite,Infinite,Infinite,Infinite,Infinite
minAF_0.05,3L,AG1000G-BF-A.Bana.2012.gambiae,22.0,210482211,0.052310,0.052201,2816.5,2580.4,3100.2,1980.7,4866.3
minAF_0.05,3L,AG1000G-BF-A.Pala.2012.coluzzii,11.0,193994771,0.120342,0.120949,Infinite,Infinite,Infinite,Infinite,Infinite
minAF_0.05,3L,AG1000G-BF-A.Pala.2012.gambiae,48.0,231821795,0.022414,0.022218,1698.3,1663.6,1734.4,745.4,Infinite
...,...,...,...,...,...,...,...,...,...,...,...
minAF_0.05,3L,AG1000G-TZ.Muleba.2015.gambiae,32.0,214584934,0.034531,0.034365,2013.0,1936.5,2095.9,1373.2,3755.5
minAF_0.05,3L,AG1000G-TZ.Tarime.2012.arabiensis,47.0,138269396,0.023323,0.022721,551.0,546.0,556.1,163.4,Infinite
minAF_0.05,3L,AG1000G-UG.Kihihi.2012.gambiae,95.0,237475490,0.011252,0.010880,892.9,888.1,897.8,601.6,1690.7
minAF_0.05,3L,AG1000G-UG.Nagongera.2012.arabiensis,81.0,136264353,0.012983,0.012832,2204.0,2159.9,2249.9,1902.8,2617.0


### Write to txt file

In [13]:
Ne.to_csv("Ne_analyses.LDNe.tsv", sep="\t", header=True, index=True, index_label='AF')

### Loading metadata

Now, lets add in all the metadata for each population, so that we can plot this on a map.

In [14]:
manifest = pd.read_csv("../../data/phase3/Ag1000g.phase3.manifest.full.tsv", sep="\t")
manifest.location = [loc.replace(" ", "") for loc in manifest.location]
manifest['pop'] = manifest[['sample_set', 'location','year', 'species_gambiae_coluzzii']].astype(str).apply('.'.join, axis=1)

We need to summarise the full manifest - essentially we want metadata info for each distinct collection we are running LDNe on, but also want AIMs and lat long info, for plotting. We first aggregate `manifest` and get the means of AIMS and long/lats, and then separately run size() on `manifest` and assign that to our new df, `manifest_summary`.

With this df, we can join the LDNe results and that will contain all we need to plot.

In [15]:
manifest_summary = manifest.groupby(['pop', 'sample_set', 'country', 'year', 'location', 'species_gambiae_coluzzii']).agg('mean').drop(columns='month').reset_index()
manifest_summary['counts'] = manifest.groupby(['pop', 'sample_set', 'year', 'location', 'species_gambiae_coluzzii']).size().reset_index(name='counts').counts

Lets write the list of pops with n > 15 to .tsv, `Phase3.LDNe.list`, so we have a list of the pops.

In [16]:
#manifest_summary[manifest_summary.counts > 8]['pop'].to_csv("../data/Phase3.LDNe.list", index=None)

In [17]:
new_manifest= manifest_summary[manifest_summary.counts > 8]

In [18]:
Ne_manifest = new_manifest.merge(Ne)

In [19]:
Ne_manifest.to_csv("Ne_manifest.tsv", sep="\t", index=None)

In [20]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(manifest_summary[manifest_summary.counts > 8])

,pop,sample_set,country,year,location,species_gambiae_coluzzii,latitude,longitude,aim_fraction_colu,aim_fraction_arab,counts
0,AG1000G-AO.Luanda.2009.coluzzii,AG1000G-AO,Angola,2009,Luanda,coluzzii,-8.884,13.302,0.937901,0.001136,81
1,AG1000G-BF-A.Bana.2012.coluzzii,AG1000G-BF-A,Burkina Faso,2012,Bana,coluzzii,11.233,-4.472,0.975738,0.002024,42
2,AG1000G-BF-A.Bana.2012.gambiae,AG1000G-BF-A,Burkina Faso,2012,Bana,gambiae,11.233,-4.472,0.035000,0.002455,22
4,AG1000G-BF-A.Pala.2012.coluzzii,AG1000G-BF-A,Burkina Faso,2012,Pala,coluzzii,11.150,-4.235,0.979909,0.002000,11
5,AG1000G-BF-A.Pala.2012.gambiae,AG1000G-BF-A,Burkina Faso,2012,Pala,gambiae,11.150,-4.235,0.027146,0.002146,48
6,AG1000G-BF-A.Souroukoudinga.2012.coluzzii,AG1000G-BF-A,Burkina Faso,2012,Souroukoudinga,coluzzii,11.235,-4.535,0.977276,0.002793,29
7,AG1000G-BF-A.Souroukoudinga.2012.gambiae,AG1000G-BF-A,Burkina Faso,2012,Souroukoudinga,gambiae,11.235,-4.535,0.028071,0.002071,28
9,AG1000G-BF-B.Bana.2014.coluzzii,AG1000G-BF-B,Burkina Faso,2014,Bana,coluzzii,11.233,-4.472,0.977532,0.002149,47
10,AG1000G-BF-B.Bana.2014.gambiae,AG1000G-BF-B,Burkina Faso,2014,Bana,gambiae,11.233,-4.472,0.025867,0.002333,15
12,AG1000G-BF-B.Pala.2014.gambiae,AG1000G-BF-B,Burkina Faso,2014,Pala,gambiae,11.150,-4.235,0.027188,0.002188,16
